# Exercise 1: Bayes Estimator and Bayes Risk

## Question 1 (M)

**Propose a supervised learning setting: input space X, output space Y, a random variable (X,Y) with a joint distribution, and a loss function l(x,y). Compute the Bayes predictor $f^*$ : X → Y and the Bayes risk associated with this setting.**

### 1. Supervised Learning Setting

*   **Input Space X:** The number of babies on an airline's planes over a 1-year period.
    *   Units: Number of babies (dimensionless).
    *   Distribution: We model $X$ as a Poisson random variable, $X \sim \mathcal{P}(\lambda)$, with $\lambda = 1,000,000$. The probability mass function is $P(X=x) = \frac{e^{-\lambda}\lambda^x}{x!}$ for $x = 0, 1, 2, ...$

*   **Output Space Y:** The number of annoyed passengers.
    *   Units: Number of annoyed passengers (dimensionless).
    *   Conditional Distribution: Given $X=x$ (the number of babies), we model $Y$ as a Binomial random variable, $Y|X=x \sim \mathcal{B}(n_x, p)$.
        *   $n_x = 10x$: The number of passengers considered "exposed" or "nearby" to babies is 10 times the number of babies.
        *   $p = 0.4$: The probability that an "exposed" passenger becomes annoyed.
    *   The probability mass function for $Y$ given $X=x$ is $P(Y=y|X=x) = \binom{n_x}{y} p^y (1-p)^{n_x-y} = \binom{10x}{y} (0.4)^y (0.6)^{10x-y}$ for $y = 0, 1, ..., 10x$.

*   **Joint Distribution P(X,Y):**
    *   $P(X=x, Y=y) = P(Y=y|X=x)P(X=x)$
    *   $P(X=x, Y=y) = \left(\binom{10x}{y} (0.4)^y (0.6)^{10x-y}\right) \cdot \left(\frac{e^{-\lambda}\lambda^x}{x!}\right)$
    *   with $\lambda = 10^6$.

*   **Loss Function $l(y_{pred}, y_{actual})$:** We choose the squared loss function.
    *   $l(y_{pred}, y_{actual}) = (y_{pred} - y_{actual})^2$

### 2. Bayes Predictor $f^*(x)$

For the squared loss function, the Bayes predictor $f^*(x)$ is the conditional expectation of $Y$ given $X=x$:
$$f^*(x) = E[Y|X=x]$$

Given $X=x$, $Y$ follows a Binomial distribution $\mathcal{B}(n_x, p)$ where $n_x = 10x$ and $p=0.4$.
The expectation of a Binomial distribution $\mathcal{B}(n,p)$ is $np$.
Therefore,
$$f^*(x) = n_x \cdot p = (10x) \cdot p$$
Substituting $p=0.4$:
$$f^*(x) = 10x \cdot 0.4 = 4x$$

The Bayes predictor is $f^*(x) = 4x$. This means the best prediction for the number of annoyed passengers is 4 times the number of babies observed.

### 3. Bayes Risk $R^*$

The Bayes risk $R^*$ is the expected value of the loss function when using the Bayes predictor $f^*(X)$. For the squared loss, this is the expected squared error:
$$R^* = \mathbb{E}\left[(f^*(X) - Y)^2\right]$$

Which implies:
$$R^* = \mathbb{E}_X\left[\mathbb{E}_{Y|X}\left[(f^*(X) - Y)^2 \mid X\right]\right]$$
But since $f^*(X) = \mathbb{E}[Y|X]$, we have:
$$R^* = \mathbb{E}_X\left[\mathbb{E}_{Y|X}\left[(\mathbb{E}[Y|X] - Y)^2 \mid X\right]\right]$$
By the equality of the conditional variance:
$$\mathbb{E}_{Y|X}\left[(\mathbb{E}[Y|X] - Y)^2 \mid X\right] = \mathbb{E}_{Y|X}\left[(Y - \mathbb{E}[Y|X])^2 \mid X\right] = \operatorname{Var}(Y|X)$$
So,
$$R^* = \mathbb{E}_X\left[\operatorname{Var}(Y|X)\right]$$

This is a general result: **the Bayes risk for the squared loss is the expected conditional variance of $Y$ given $X$**.

Now, for our specific model:

Given $X=x$, $Y \sim \mathcal{B}(n_x, p)$ with $n_x = 10x$ and $p=0.4$.
The variance of a Binomial distribution $\mathcal{B}(n,p)$ is $np(1-p)$.
So,
$$\operatorname{Var}(Y|X=x) = n_x \cdot p \cdot (1-p) = (10x) \cdot 0.4 \cdot 0.6 = 2.4x$$

Now we compute the expectation of this conditional variance over the distribution of $X$:
$$R^* = \mathbb{E}_X[2.4X] = 2.4 \cdot \mathbb{E}[X]$$
The expectation of a Poisson distribution $X \sim \mathcal{P}(\lambda)$ is $\mathbb{E}[X] = \lambda$.
Given $\lambda = 1,000,000$:
$$R^* = 2.4 \cdot \lambda = 2.4 \cdot 1,000,000 = 2,400,000$$

**Conclusion for Question 1:**

*   **Input Space X:** Number of babies, $X \sim \mathcal{P}(\lambda=10^6)$.
*   **Output Space Y:** Number of annoyed passengers. Conditional on $X=x$, $Y|X=x \sim \mathcal{B}(n_x=10x, p=0.4)$.
*   **Loss Function:** Squared loss, $l(y_{pred}, y_{actual}) = (y_{pred} - y_{actual})^2$.
*   **Bayes Predictor $f^*(x)$:** $f^*(x) = 4x$.
*   **Bayes Risk $R^*$:** $R^* = 2.4\lambda = 2,400,000$.


## Question 2 (C): Simulation comparing estimators

**Propose an estimator $\tilde{f} : X → Y$, different than the Bayes estimator and run a simulation that gives a statistical approximation of its generalization error by computing its empirical risk on a test set. Perform the same simulation also for $f^*$, and verify that the generalization error is smaller for $f^*$ than for $\tilde{f}$, and that your test error for $f^*$ is close to the Bayes risk.**

We will compare:
- **Bayes estimator**: $f^*(x) = 4x$ 
- **Alternative estimator**: $\tilde{f}(x) = 5x$ (hypothesizing that each baby annoys 5 passengers, or 1 out of 2 exposed passengers)
- **Bayes Risk $R^*$:** $R^* = 2.4\lambda = 2,400,000$.

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import poisson, binom

# Set random seed for reproducibility
np.random.seed(42)

# Parameters from our model
lambda_param = 1000000  # Parameter for Poisson distribution of X (number of babies)
p = 0.4  # Probability that an exposed passenger is annoyed
n_test_samples = 10000  # Number of test samples

print("=== Exercise 1, Question 2: Simulation ===")
print(f"Model parameters: λ = {lambda_param}, p = {p}")
print(f"Theoretical Bayes risk: {2.4 * lambda_param:,.0f}")

# Generate test data
print(f"\nGenerating {n_test_samples:,} test samples...")

# Sample X from Poisson(λ)
X_test = poisson.rvs(mu=lambda_param, size=n_test_samples)

# For each X=x, sample Y from Binomial(n=10x, p=0.4)
Y_test = np.zeros(n_test_samples)
for i in range(n_test_samples):
    n_x = 10 * X_test[i]
    Y_test[i] = binom.rvs(n=n_x, p=p)

print(f"Test data generated. X range: [{X_test.min():,}, {X_test.max():,}]")
print(f"Y range: [{Y_test.min():,.0f}, {Y_test.max():,.0f}]")

# Define estimators
def bayes_estimator(x):
    """Bayes estimator: f*(x) = 4x"""
    return 4 * x

def mean_estimator(x):
    """Alternative estimator: f_tilde(x) = 5x"""
    return 5 * x

# Compute predictions
pred_bayes = bayes_estimator(X_test)
pred_mean = mean_estimator(X_test)  # Now f_tilde(x) = 5x

# Compute squared losses
loss_bayes = (pred_bayes - Y_test) ** 2
loss_mean = (pred_mean - Y_test) ** 2

# Compute empirical risks (average squared loss)
risk_bayes = np.mean(loss_bayes)
risk_mean = np.mean(loss_mean)

print(f"\n=== Results ===")
print(f"Empirical risk for Bayes estimator f*(x) = 4x: {risk_bayes:,.0f}")
print(f"Empirical risk for alternative estimator f_tilde(x) = 5x: {risk_mean:,.0f}")
print(f"Theoretical Bayes risk: {2.4 * lambda_param:,.0f}")

print(f"\nVerification:")
print(f"✓ Bayes estimator has lower risk, and is off by : {abs(risk_bayes - 2.4 * lambda_param) / (2.4 * lambda_param) * 100:.2f}%")
print(f"✓ Alternative estimator has higher risk, and is off by: {abs(risk_mean - 2.4 * lambda_param) / (2.4 * lambda_param) * 100:.2f}%")
print("Note that with the square loss on values the order of 10^6, the differences become very big.")

=== Exercise 1, Question 2: Simulation ===
Model parameters: λ = 1000000, p = 0.4
Theoretical Bayes risk: 2,400,000

Generating 10,000 test samples...
Test data generated. X range: [996,493, 1,003,890]
Y range: [3,984,066, 4,019,024]

=== Results ===
Empirical risk for Bayes estimator f*(x) = 4x: 2,341,663
Empirical risk for alternative estimator f_tilde(x) = 5x: 1,000,001,999,590
Theoretical Bayes risk: 2,400,000

Verification:
✓ Bayes estimator has lower risk, and is off by : 2.43%
✓ Alternative estimator has higher risk, and is off by: 41666649.98%
Note that with the square loss on values the order of 10^6, the differences become very big.
